In [1]:
import requests
import re
import pymorphy2
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from fake_useragent import UserAgent
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import gensim.downloader as api
from gensim.models import Word2Vec

In [2]:
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

In [3]:
from nltk import ngrams
from pymystem3 import Mystem

In [29]:
tag_to_tag = {
    'SPRO': 'PRON', 'ADVPRO': 'ADV', 'APRO': 'PRON',
    'NPRO': 'PRON', 'DET': 'PRON', 'S': 'NOUN',
    'PROPN': 'NOUN', 'V': 'VERB', 'INFN': 'VERB',
    'PRTS': 'PRT', 'PRTF': 'PRT', 'ADVB': 'ADV',
    'ANUM': 'NUM', 'NUMR': 'NUM', 'ADJF': 'ADJ',
    'ADJS': 'ADJ', 'PR': 'ADP', 'CCONJ': 'CONJ',
    'SCONJ': 'CONJ', 'PRCL': 'PART', 'AUX': 'VERB',
    'PREP': 'ADP', 'A': 'ADJ'
}
is_cyrilic = re.compile(r'[а-яА-ЯёЁ]+?\b')

def mystem_tag(sent):
    m = Mystem()
    poses = []
    analysis = m.analyze(sent)
    for word in analysis:
        if is_cyrilic.match(word['text']):
            try:
                feats = word['analysis'][0]['gr']
                pos = feats.split('=')[0].split(',')[0]
            except:
                pos = 'X'
            if pos == 'V':
                if 'деепр' in feats:
                    pos = 'GRND'
                elif 'прич' in feats:
                    pos = 'PRT'
            if pos in ['A', 'ADV']:
                if 'срав' in feats:
                    pos = 'COMP'
            if pos in tag_to_tag.keys():
                poses.append(tag_to_tag[pos])
            else:
                poses.append(pos)
    return poses

def chunker(pattern, text):
    m = Mystem()
    result = []
    pattern = pattern.split('+')
    lemmas = [lemma for lemma in m.lemmatize(text) if is_cyrilic.match(lemma)]
    poses = mystem_tag(text)
    result_sentence = [(l, p) for l, p in zip(lemmas, poses)]
    all_ngrams = ngrams(result_sentence, len(pattern))
    word_ngrams = list(ngrams(lemmas, len(pattern)))
    if len(lemmas) != len(poses):
        return []
    for j, ngram in enumerate(all_ngrams):
        cnt = 0
        for i, word in enumerate(pattern):
            if word in ngram[i]:
                cnt += 1
        if cnt == len(pattern):
            result.append(' '.join(word_ngrams[j]))
    return result

### 1. Сбор данных

Я выбрала сайт https://www.turpravda.com/tn/top-hotels.html с отзывами на отели Туниса. Оттуда я взяла страницы с отелями, которые оценивались на 9-10, 7, 6 и 1-5 баллов в среднем. На каждой странице максимум 25 отелей. 

Так как страницы с самими отзывами позволяют просматривать новые отзывы только по нажатию специальной кнопки, пришлось брать только по 10 первых отзывов с каждой страницы, так как я не нашла функционала, с помощью которого можно достать больше отзывов.

In [5]:
def get_comments(page_url):
    global df_comments
    
    session = requests.session()
    req = session.get(page_url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page)
    
    for comment in soup.find_all('div', {'class': 'ans_body'}):
        mark = comment.find('span', {'class': 'value'})
        if mark:
            comment_text = comment.find('span', {'class': 'all-text'}).text
            mark = float(mark.text[-4:])
            df_comments = df_comments.append({'comment': comment_text, 
                                              'mark': mark,
                                              'url': page_url}, ignore_index=True)

In [6]:
df_comments = pd.DataFrame(columns=['comment', 'mark', 'url'])
rates = [9, 7, 5, '5&p=2']

for i in rates:
    page_url = f'https://www.turpravda.com/tn/top-hotels.html?rte%5B%5D={i}'
    session = requests.session()
    req = session.get(page_url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page)

    all_hrefs = soup.find_all('a', {'class': 'hotel-name-title'})
    for href in tqdm(all_hrefs[:6]):
        link = href.get('href')
        get_comments('https://www.turpravda.com' + link)

100%|██████████| 6/6 [00:08<00:00,  1.38s/it]


Таким образом, у меня получилось почти равное количество положительных и отрицательных отзывов. К отрицательным я относила те отзывы, в которых оценка от 1 до 5, включительно. Остальные я относила к положительным.

In [7]:
df_comments['sentiment'] = df_comments['mark'].apply(lambda x: 1 if x > 5 else 0)
df_comments['sentiment'].value_counts()

1    111
0     86
Name: sentiment, dtype: int64

### 2. Создание словаря

In [9]:
morph = pymorphy2.MorphAnalyzer()
sw = stopwords.words('russian')
def clean_text(text):
    tokens = []
    for word in word_tokenize(text):
        if is_cyrilic.search(word):
            if word not in sw:
                tokens.append(morph.parse(word.lower())[0].normal_form)
    return tokens

def clean_text_not_sw(text):
    tokens = []
    for word in word_tokenize(text):
        if is_cyrilic.search(word):
            tokens.append(morph.parse(word.lower())[0].normal_form)
    return tokens

In [10]:
df_comments['tokens'] = df_comments['comment'].apply(clean_text)
df_comments['clean_comment'] = df_comments['tokens'].apply(lambda x: ' '.join(x))

In [11]:
X = df_comments['comment']
y = df_comments['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=8)

In [13]:
mask_pos = y==1
X_train = pd.Series(X_train)
positive_comments = X_train[mask_pos].tolist()
negative_comments = X_train[~mask_pos].tolist()

positive_corpus = []
negative_corpus = []

for t in positive_comments:
    tokens = clean_text(t)
    positive_corpus.extend(tokens)
for t in negative_comments:
    tokens = clean_text(t)
    negative_corpus.extend(tokens)

### Применение чанкера

Я выбрала три шаблона: 'быть+ADJ', 'ADJ+еда', 'не+быть+NOUN'. Я предположила, что они могут помочь определиться с тональностью, так как часто встречаются в отзывах и обычно имеют определённый окрас, не нейтральны. Например, _был чистым_, _отвратительная еда_, _не было полотенец_.

In [14]:
bigrams_pos = []
bigrams_neg = []
patterns = ['быть+ADJ', 'ADJ+еда', 'не+быть+NOUN']
for pattern in patterns:
    for comment in tqdm(positive_comments):
        new_tokens = chunker(pattern, comment)
        bigrams_pos.extend(new_tokens)

    for comment in tqdm(negative_comments):
        new_tokens = chunker(pattern, comment)
        bigrams_neg.extend(new_tokens)
    print(len(bigrams_neg), len(bigrams_pos))

100%|██████████| 70/70 [02:53<00:00,  2.48s/it]


34 51


100%|██████████| 70/70 [02:53<00:00,  2.47s/it]


42 59


100%|██████████| 70/70 [02:49<00:00,  2.42s/it]

45 73


In [23]:
cnt_pos = Counter(positive_corpus).most_common(200)
cnt_neg = Counter(negative_corpus).most_common(200)

set_pos = set(dict(cnt_pos).keys())
set_pos_all = set_pos.union(set(bigrams_pos))
set_neg = set(dict(cnt_neg).keys())
set_neg_all = set_neg.union(set(bigrams_neg))
print('Only positive:')
print(set_pos-set_neg)
print('Only negative:')
print(set_neg-set_pos)

Only positive:
{'рыба', 'кто', 'ещё', 'душа', 'красивый', 'язык', 'английский', 'этот', 'бесплатно', 'отличный', 'ездить', 'расположить', 'приятный', 'немного', 'сахар', 'супер', 'возле', 'хотеться', 'новый', 'впечатление', 'вечер', 'центр', 'прекрасный', 'правда', 'дорога', 'зелёный', 'медина', 'сейф', 'решить', 'какой-то', 'египет', 'купить', 'чаевой', 'больший', 'небольшой', 'у', 'увидеть', 'замечательный', 'огромный', 'оставить', 'около', 'д.', 'аэропорт', 'ваш', 'доллар', 'ждать', 'отдельный', 'поменять', 'покупать', 'конец', 'страна', 'любой', 'знать', 'вопрос', 'мясо', 'кормить', 'достаточно', 'поездка', 'кухня', 'аниматор', 'шоу', 'город', 'турция', 'кстати'}
Only negative:
{'за', 'приличный', 'заселение', 'неделя', 'поздний', 'второй', 'кондиционер', 'рубль', 'стакан', 'к.', 'рекомендовать', 'ресепшен', 'бельё', 'утром', 'быстро', 'сутки', 'нормально', 'стол', 'лежать', 'тарелка', 'детский', 'минус', 'тур', 'делать', 'кофе', 'ужасный', 'комната', 'постельный', 'мусор', 'але', 

In [24]:
def sentiment(comments, pos_dict, neg_dict):
    cnt_pos = 0
    cnt_neg = 0
    result = []
    for i, comment in enumerate(comments):
        tokens = clean_text_not_sw(comment)
        ngr = list(ngrams(tokens, 2))
        tokens.extend(ngr)
        for token in tokens:
            if token in pos_dict:
                cnt_pos += 1
            elif token in neg_dict:
                cnt_neg += 1
        if cnt_neg > cnt_pos:
            result.append(0)
        else:
            result.append(1)
        cnt_pos = 0
        cnt_neg = 0
    return result

In [25]:
def accuracy_count(y_pred, y_test):
    cnt = 0
    for p, t in zip(y_pred, y_test):
        if p == t:
            cnt += 1
    return cnt / len(y_pred)

In [26]:
only_pos = set_pos - set_neg
only_neg = set_neg - set_pos
y_pred = sentiment(X_test, only_pos, only_neg)
accuracy_count(y_pred, y_test)

0.725

In [27]:
only_pos = set_pos_all - set_neg_all
only_neg = set_neg_all - set_pos_all
y_pred = sentiment(X_test, only_pos, only_neg)
accuracy_count(y_pred, y_test)

0.725

Но результат не изменился, скорее всего из-за того, что отзывов мало, следовательно, маленькая вероятность, что встретятся одинаковые н-граммы.